In [1]:
import numpy as np

temp =np.load('threshold.npy')

In [2]:
print(temp)

[[  1  13]
 [  7  58]
 [  1  26]
 [  1  26]
 [  1  22]
 [  8 100]]


In [3]:
import pandas as pd

df = pd.read_csv('sort_data.csv')

In [4]:
df.iloc[:,8:].shape

(170, 10)

In [5]:
df.columns[8:]

Index(['CCNA_00001', 'CCNA_00002', 'CCNA_00003', 'CCNA_00004', 'CCNA_00005',
       'CCNA_00006', 'CCNA_00007', 'CCNA_00008', 'CCNA_00009', 'CCNA_00010'],
      dtype='object')

In [6]:
import numpy as np
np.asarray(df.groupby(['strain', 'condition']).size())

array([11, 12,  7, 12, 12, 12,  7, 11, 12, 12,  7, 12, 12, 12,  7, 12])

In [7]:
df.groupby(['strain', 'condition']).size()

strain     condition         
DCLPA      canavanine            11
           heat                  12
           none                   7
           oxidative-peroxide    12
DLON       canavanine            12
           heat                  12
           none                   7
           oxidative-peroxide    11
dnak-dnaJ  canavanine            12
           heat                  12
           none                   7
           oxidative-peroxide    12
wild-type  canavanine            12
           heat                  12
           none                   7
           oxidative-peroxide    12
dtype: int64

In [8]:
L = 10
df_temp = pd.read_csv('temp.csv')
df_p = df_temp.nlargest(100, 'gij_1')
df_p[df_p.columns[1:len(df_p.columns)]] = df_p[df_p.columns[1:len(
    df_p.columns)]].apply(lambda x: x / sum(x))
df_p = df_p.sort_values('gij_1', ascending=False)
df_p.to_csv('inference_5.csv', index=False)

H_matrix = np.zeros((5,L), dtype='int')
H_matrix_str = np.asarray(df_p['H'])
for i in range(5):
    H_matrix_str[i] = H_matrix_str[i].replace(", ", "").replace("[", "").replace("]", "")

for i in range(5):
    for j in range(L):
        if H_matrix_str[i][j] == "0":
            H_matrix[i,j] = int(0)
        if H_matrix_str[i][j] == "1":
            H_matrix[i,j] = int(1)

df_H = pd.DataFrame(H_matrix.T)
locus_tag = df.columns[8:]
df_H["locus_tag"] = locus_tag
df_H = df_H[["locus_tag", 0, 1, 2, 3, 4]]
df_H.columns = ["locus_tag", "top_1", "top_2", "top_3", "top_4", "top_5"]
df_H.to_csv('H_matrix.csv', index=False)

In [19]:
import numpy as np

temp = np.load('threshold.npy')

In [22]:
temp[0]

array([ 0, 21])

In [24]:
temp[4]

array([ 0, 10])

In [6]:
import numpy as np
import glob
import pandas as pd

from path import Path


datafile = 'peter_UC_processed_'
samples = glob.glob(datafile + '_psamples/*state*')
print('%d posterior samples' % len(samples))
df_data = pd.read_csv('peter_UC_processed.csv')
n_biopsies_I = np.asarray(df_data.groupby(['strain','condition']).size())


H = []
G = []
Gi = []
Gij = []
L = 4084
total_sample = sum(n_biopsies_I)
# threshold = 0.0005
n_psample = len(samples)

for s in range(len(samples)):
    sample_file = np.load(samples[s])  # load in sample
    # this is the H matrix (using paper notation), sorry for different notation
    h = sample_file['Z_LK'].T
#    print(Z_LK.shape)          # this is a loci x component matrix

    Theta_K = sample_file['Theta_K']   # unnormalized G distribution
    # unnormalized per-individual G distributions
    Theta_IK = sample_file['Theta_IK']
    # unnormalized per-sample G distributions
    Theta_SK = sample_file['Theta_SK']

    # normalized G distribution
    g = Theta_K / Theta_K.sum()
    # normalized per-individual G distributions
    gi = Theta_IK / np.sum(Theta_IK, axis=1, keepdims=True)
    # normalized per-sample G distributions
    gij = Theta_SK / np.sum(Theta_SK, axis=1, keepdims=True)

    H.append(h)
    G.append(g)
    Gi.append(gi)
    Gij.append(gij)

flattern = [str(s.tolist()) for item in H for s in item]

glist = [s for item in G for s in item]
# glist = [s for item in G[-1] for s in item]



gilist = []
for i in range(len(n_biopsies_I)):
    flattern_gi = [s for item in Gi for s in item[i]]
    gilist.append(flattern_gi)


gijlist = []
for i in range(total_sample):
    flattern_gij = [s for item in Gij for s in item[i]]
    gijlist.append(flattern_gij)


df = pd.DataFrame({'H': flattern})

df['g0'] = glist

for i in range(len(gilist)):
    df['gi_'+str(i+1)] = gilist[i]

for i in range(len(gijlist)):
    df['gij_'+str(i+1)] = gijlist[i]

df_temp = df.groupby(['H']).agg('sum').reset_index()
print('Done!')
df_temp.to_csv('temp_1.csv', index=False)

1001 posterior samples
Done!


In [109]:
path_name_2 = 'results/' + datafile
out_dir = Path(path_name_2)
out_dir.makedirs_p()

# df_p = df_temp.nlargest(5, 'g0')
df_temp[df_temp.columns[1:len(df_temp.columns)]] = df_temp[df_temp.columns[1:len(
    df_temp.columns)]].apply(lambda x: x / sum(x))

In [114]:
threshold = 0.0005
df_p = df_temp[(df_temp[df_temp.columns[-(total_sample):]]
                > threshold).any(axis=1)]

df_p[df_p.columns[1:len(df_p.columns)]] = df_p[df_p.columns[1:len(
    df_p.columns)]].apply(lambda x: x / sum(x))

df_p = df_p.sort_values('g0', ascending=False)

row = ['H'] + ['Top_Level'] + list(df_data.groupby(['strain', 'condition']).groups.keys()) + list(df_data.groupby(['strain', 'condition', 'slevel', 'rep', 'batch', 'time']).groups.keys())
row_condition = pd.DataFrame([row], columns=list(df_p.columns))

df_annote =  pd.concat([row_condition, df_p])

df_annote.to_csv('inference_oak_1.csv', index=False)

/home/shaihe/.conda/envs/hgp/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [115]:
locus_att = pd.read_csv('locus_attribute.tab', sep='\t')
locus_att = locus_att[['locus_tag', 'Name', 'description']]
locus_tag = df_data.columns[7:]
df_locus_tag = pd.DataFrame(locus_tag)
df_locus_tag.columns = ['locus_tag']
df_att = df_locus_tag.merge(locus_att, on='locus_tag', how='left')

In [116]:
n_cluster = len(df_p)
H_matrix = np.zeros((n_cluster,L), dtype='int')
H_matrix_str = np.asarray(df_p['H'])
for i in range(n_cluster):
    H_matrix_str[i] = H_matrix_str[i].replace(", ", "").replace("[", "").replace("]", "")

for i in range(n_cluster):
    for j in range(L):
        if H_matrix_str[i][j] == "0":
            H_matrix[i,j] = int(0)
        if H_matrix_str[i][j] == "1":
            H_matrix[i,j] = int(1)

df_H = pd.DataFrame(H_matrix.T)
df_H["locus_tag"] = locus_tag
df_H['Gene_Name'] = df_att['Name']
df_H['Description'] = df_att['description']

cols = list(df_H.columns)
cols = [cols[-3]] + [cols[-2]] + [cols[-1]] + cols[:-3]
df_H = df_H[cols]

# df_H = df_H[["locus_tag", 0, 1, 2, 3, 4]]
# df_H.columns = ["locus_tag", "top_1", "top_2", "top_3", "top_4", "top_5"]

df_H.to_csv('H_matrix_oak_1.csv', index=False)